In [ ]:
import pandas as pd
import yfinance as yf
import itertools
from tqdm import tqdm
import random
import statsmodels.api as sm
import numpy as np
from ast import literal_eval

# Data Extraction

In [ ]:
TRAIN_START_DATE = "2023-01-01"
TRAIN_END_DATE = "2023-12-31"

All tickers data are obtained from: https://www.nasdaq.com/market-activity/stocks/screener

In [ ]:
ticker_df = pd.read_csv("nasdaq_screener.csv")
# dropna
ticker_df = ticker_df[~ticker_df["Symbol"].isna()]
ticker_df["Sector"] = ticker_df["Sector"].str.replace(" ", "-")
ticker_df.head(1)

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,AACG,ATA Creativity Global American Depositary Shares,$0.6507,0.0056,0.868%,20822485.0,China,2008.0,153860,Real-Estate,Other Consumer Services


Number of stocks in each sector

In [ ]:
ticker_df[["Sector", "Name"]].groupby("Sector").count().reset_index().sort_values("Name", ascending=False)

,Sector,Name
5,Health-Care,1046
4,Finance,885
1,Consumer-Discretionary,669
9,Technology,583
6,Industrials,284
2,Consumer-Staples,92
8,Real-Estate,88
10,Telecommunications,67
3,Energy,54
11,Utilities,51


In [ ]:
ticker_list = ticker_df["Symbol"].to_list()
sector_list = ticker_df["Sector"].to_list()

ticker_sector_dict = {}
for idx in range(len(ticker_list)):
  ticker_sector_dict[ticker_list[idx]] = sector_list[idx]

Tickers' closing price obtained from Yahoo Finance

In [ ]:
price_df = pd.DataFrame()
print("Downloading train set NASDAQ prices.")
# Fetch the historical market data for the tickers
price_df = yf.download(ticker_list, start=TRAIN_START_DATE, end=TRAIN_END_DATE)['Close']
# Drop tickers with missing prices
price_df = price_df.dropna(axis=1)
#price_df.to_csv("nasdaq_price.csv")

[**********************70%%********              ]  2786 of 3976 completed

$MFICL: possibly delisted; No price data found  (1d 2023-01-01 -> 2023-12-31)


[*********************100%%**********************]  3976 of 3976 completed

439 Failed downloads:
['VRMEW', 'CRESW', 'TRONW', 'NXPLW', 'BFRGW', 'HTZWW', 'EMLDW', 'KPLTW', 'HUMAW', 'YOTAW', 'EDBLW', 'IXAQW', 'REVBW', 'PTWOW', 'WINVR', 'BFIIW', 'DISTW', 'PAYOW', 'RVMDW', 'CEADW', 'ECDAW', 'NXGLW', 'ADSEW', 'TMTCR', 'COEPW', 'ASTSW', 'FNVTW', 'DFLIW', 'RVPHW', 'OCSAW', 'VMCAW', 'ABLVW', 'RFAIR', 'NVACR', 'LSBPW', 'AILEW', 'PLMIW', 'AUROW', 'MRNOW', 'AIMAW', 'OAKUW', 'PAVMZ', 'ADNWW', 'CLNNW', 'MACIW', 'RCKTW', 'PCTTW', 'ONMDW', 'HUBCW', 'MMVWW', 'USGOW', 'SXTPW', 'CETUR', 'BUJAW', 'WGSWW', 'ACACW', 'BAERW', 'HYMCW', 'INTEW', 'FREEW', 'SATLW', 'BCDAW', 'ABVEW', 'SRZNW', 'CXAIW', 'GRDIW', 'OXBRW', 'ZAPPW', 'CETUW', 'KITTW', 'GDSTW', 'EVLVW', 'CCTSW', 'MTEKW', 'BNAIW', 'SMXWW', 'SHMDW', 'SCLXW', 'KWESW', 'MACAW', 'CIFRW', 'UKOMW', 'XOSWW', 'CPTNW', 'BCSAW', 'MDAIW', 'SVIIW', 'RGTIW', 'BOWNR', 'JSPRW', 'GCMGW', 'PFTAW', 'COOTW', 'DBGIW', 'QSIAW', 'AFARW', 'FIACW', 'TNONW', 'CEROW', 'CGBSW', '

In [ ]:
price_df.head()

Ticker,AACG,AACI,AACIU,AADI,AAGR,AAL,AAME,AAOI,AAON,AAPL,...,ZLS,ZM,ZNTL,ZS,ZTEK,ZUMZ,ZVRA,ZVSA,ZYME,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-03,1.26,10.13,10.12,12.31,7.096774,12.74,2.42,1.88,49.886665,125.070000,...,10.00,66.660004,20.650000,110.190002,1.55,22.010000,4.53,605.500000,7.54,13.88
2023-01-04,1.27,10.14,10.12,12.67,7.139785,13.59,2.43,1.92,47.779999,126.360001,...,10.01,67.650002,21.040001,111.589996,1.53,22.580000,4.73,542.500000,7.67,14.11
2023-01-05,1.32,10.15,10.14,12.52,7.146953,13.99,2.47,1.88,47.000000,125.019997,...,10.02,65.620003,21.270000,104.389999,1.50,22.370001,4.48,567.000000,8.00,14.14
2023-01-06,1.41,10.16,10.13,12.49,7.157706,14.18,2.44,1.86,49.459999,129.619995,...,10.02,69.510002,21.209999,103.959999,1.48,23.309999,4.57,563.500000,7.65,15.65
2023-01-09,1.40,10.16,10.13,12.13,7.168459,14.61,2.58,2.15,49.606667,130.149994,...,10.05,70.080002,18.530001,104.750000,1.45,23.540001,4.54,861.349976,7.54,15.65


In [ ]:
# updating the ticker list as some of the tickers were dropped due to missing data
ticker_list = list(price_df.columns)[1:]

# Regression on three stocks

Let's suppose we want to have 3 stocks in our portfolio, we compute spreads (the white noise) by regressing two stocks on the other stocks. We then compute the spreads, r-squared and its fitted parameters.

In this case, we are using Ordinary Least Squared (OLS) fit, hence we should expect a good portfolio  should have spread with mean 0.

A nice spread to trade is when
1. It's stationary
2. it standard deviation is not too high (less risk).

This can be perfectly identified with R-squared (R2). This is because when r-squared is higher, the more stable the spread is.

In the following section, we randomly choose 3 stocks for n times, and identified the best combinations (i.e. with the highest r-squared and it passes stationary test).

In [ ]:
# Regression on three stock prices (using OLS
def regression_on_three_stocks(df):
    X = df.iloc[:,:-1]
    Y = df.iloc[:,-1]
    X = sm.add_constant(X)
    model = sm.OLS(Y, X).fit()
    spread = Y - (model.params[1] * df.iloc[:,0] + model.params[2] * df.iloc[:,1] + model.params[0])
    return model.rsquared, spread, model.params, model.pvalues

In [ ]:
def within_coefficients_scale(coefficient1, coefficient2, scale_limit):
    # coefficient of Y, X1(beta1), X2(beta2)
    coefficients_abs = [1, np.abs(coefficient1), np.abs(coefficient2)]
    return (max(coefficients_abs) / min(coefficients_abs)) <= scale_limit

In [ ]:
train_df = price_df
healthcare_tickerlist = [col for col in train_df.columns if col in ticker_df.loc[ticker_df["Sector"]=="Health-Care", "Symbol"].to_list()]
finance_tickerlist = [col for col in train_df.columns if col in ticker_df.loc[ticker_df["Sector"]=="Finance", "Symbol"].to_list()]
consumer_tickerlist = [col for col in train_df.columns if col in ticker_df.loc[ticker_df["Sector"]=="Consumer-Discretionary", "Symbol"].to_list()]
technology_tickerlist = [col for col in train_df.columns if col in ticker_df.loc[ticker_df["Sector"]=="Technology", "Symbol"].to_list()]
industrials_tickerlist = [col for col in train_df.columns if col in ticker_df.loc[ticker_df["Sector"]=="Industrials", "Symbol"].to_list()]

# Stock Selection Criteria

In [ ]:
# filtering
weightage_cutoff = 0.2
scale_max_limit = 20

def apply_filter_conditions(df):
  condition_1 = (df["rsqaured"] > 0.9)
  condition_2 = (df["param_1"].abs() > weightage_cutoff)
  condition_3 = (df["param_2"].abs() > weightage_cutoff)
  condition_4 = df.apply(lambda row: within_coefficients_scale(row["param_1"], row["param_2"], scale_max_limit), axis=1)

  return df[condition_1 & condition_2 & condition_3 & condition_4]

In [ ]:
## Loop through all possible 3 combination of stocks to find best r-squared, by randomly picking 3 combinations 100 times
def get_best_stocks(df, tickers, num_samples):
  """
  1. Get combinations of all tickers
  2. For each ticker combination measure the R-square

  *Note: the last ticker of ticker_combi_selected is the response variable

  tickers: list() of tickers
  """
  rsquared = []
  spreads = []
  param_0 = []
  param_1 = []
  param_2 = []
  triplets = []

  # get combinations
  ticker_combi = list(itertools.combinations(tickers, 3))

  # sample only num_samples
  idxs = random.sample(range(min(len(ticker_combi), num_samples)), num_samples)
  ticker_combi_selected = [ticker_combi[idx] for idx in idxs]

  # check if regression coefficient is significantly > 0 at 5% confidence level
  pvalue_cutoff = 0.05

  for stocks in tqdm(ticker_combi_selected):
      r, spread, param, pvalues = regression_on_three_stocks(df[list(stocks)])
      if np.all(np.abs(pvalues[1:]) < pvalue_cutoff):
        triplets.append(list(stocks))
        rsquared.append(r)
        spreads.append(spread)
        param_0.append(param[0])
        param_1.append(param[1])
        param_2.append(param[2])

  return pd.DataFrame({
      "tickers":triplets,
      "rsqaured":rsquared,
      "spreads":spreads,
      "param_0":param_0,
      "param_1":param_1,
      "param_2":param_2,
  })

In [ ]:
technology_df = get_best_stocks(train_df, technology_tickerlist, 10000)
technology_df = apply_filter_conditions(technology_df)
technology_df = technology_df.sort_values("rsqaured", ascending=False)
technology_df = technology_df.reset_index(drop=True)
technology_df['sector'] = 'Technology'
technology_df

,tickers,rsqaured,spreads,param_0,param_1,param_2,sector
0,"(AAOI, AAPL, TTWO)",0.941185,Date 2023-01-03 6.929834 2023-01-04 9.65...,6.877691,0.837044,0.701438,Technology
1,"(AAOI, AIRG, PHUN)",0.936302,Date 2023-01-03 -1.832283 2023-01-04 -0.45...,-10.648280,-0.638324,7.967737,Technology
2,"(AAOI, AAPL, LRCX)",0.930703,Date 2023-01-03 -5.346900 2023-01-04 -1....,41.781284,7.707764,2.905373,Technology
3,"(AAOI, AAPL, MSFT)",0.915438,Date 2023-01-03 25.766030 2023-01-04 12....,-34.524471,1.266108,1.966564,Technology
4,"(AAOI, ADBE, INTU)",0.914145,Date 2023-01-03 -14.589959 2023-01-04 -15....,280.351750,5.028369,0.344191,Technology
5,"(AAOI, AAPL, WDAY)",0.910720,Date 2023-01-03 7.027300 2023-01-04 7....,64.935802,3.010795,0.733402,Technology
6,"(AAOI, AAPL, AVGO)",0.910591,Date 2023-01-03 6.645730 2023-01-04 6.66...,-12.330085,1.390881,0.467078,Technology
7,"(AAOI, AIRG, CMBM)",0.910055,Date 2023-01-03 1.982433 2023-01-04 1.28...,-1.235152,-0.231892,3.120394,Technology
8,"(AAOI, ACMR, CAMT)",0.905518,Date 2023-01-03 -1.638663 2023-01-04 -1.79...,-5.555299,1.112520,2.861711,Technology
9,"(AAOI, AAPL, MANH)",0.903581,Date 2023-01-03 8.776508 2023-01-04 6....,-46.103889,1.595266,1.241931,Technology


In [ ]:
finance_df = get_best_stocks(train_df, finance_tickerlist, 10000)
finance_df = apply_filter_conditions(finance_df)
finance_df = finance_df.sort_values("rsqaured", ascending=False)
finance_df = finance_df.reset_index(drop=True)
finance_df['sector'] = 'Finance'
finance_df

,tickers,rsqaured,spreads,param_0,param_1,param_2,sector
0,"(AACI, ACAC, TLGY)",0.972664,Date 2023-01-03 0.027596 2023-01-04 0.00...,-0.542464,0.277815,0.780880,Finance
1,"(AACI, ABCB, MSBI)",0.969617,Date 2023-01-03 0.895914 2023-01-04 0.88...,24.463299,-1.754739,0.409207,Finance
2,"(AACI, ABCB, PNFP)",0.966421,Date 2023-01-03 -1.483943 2023-01-04 -0.35...,-95.398458,9.353662,1.586173,Finance
3,"(AACI, ABCB, HOPE)",0.964588,Date 2023-01-03 0.547334 2023-01-04 0.39...,26.294895,-2.548562,0.257769,Finance
4,"(AACI, ABCB, CBAN)",0.963773,Date 2023-01-03 0.520061 2023-01-04 0.45...,15.960681,-1.317442,0.209826,Finance
...,...,...,...,...,...,...,...
130,"(AACI, ACT, NMIH)",0.901367,Date 2023-01-03 -2.229921 2023-01-04 -2.46...,-16.678630,1.534020,1.013184,Finance
131,"(AACI, ABCB, FRST)",0.901018,Date 2023-01-03 0.185533 2023-01-04 0.21...,20.798693,-1.999824,0.238747,Finance
132,"(AACI, ABCB, STBA)",0.900593,Date 2023-01-03 -1.082561 2023-01-04 -1.01...,89.889205,-7.539404,0.466469,Finance
133,"(AACI, AAME, SLAM)",0.900220,Date 2023-01-03 -0.192720 2023-01-04 -0.18...,7.355620,0.409598,-0.496748,Finance


In [ ]:
healthcare_df = get_best_stocks(train_df, healthcare_tickerlist, 10000)
healthcare_df = apply_filter_conditions(healthcare_df)
healthcare_df = healthcare_df.sort_values("rsqaured", ascending=False)
healthcare_df = healthcare_df.reset_index(drop=True)
healthcare_df['sector'] = "Health-Care"
healthcare_df

,tickers,rsqaured,spreads,param_0,param_1,param_2,sector
0,"(AADI, ACET, DTIL)",0.955101,Date 2023-01-03 -1.200676 2023-01-04 4.10...,2.069040,1.465539,1.889247,Health-Care
1,"(AADI, ABCL, PYPD)",0.948678,Date 2023-01-03 -1.008345 2023-01-04 -2.61...,-9.088195,1.302905,1.732115,Health-Care
2,"(AADI, ABUS, ENTA)",0.945528,Date 2023-01-03 5.817433 2023-01-04 4.69...,-40.737716,3.238936,18.295504,Health-Care
3,"(AADI, ABVC, NVCR)",0.944882,Date 2023-01-03 -8.408477 2023-01-04 -8....,-9.491445,3.692205,6.731479,Health-Care
4,"(AADI, ABUS, SILK)",0.944379,Date 2023-01-03 5.817518 2023-01-04 4.47...,-38.675920,3.431719,18.212527,Health-Care
...,...,...,...,...,...,...,...
98,"(AADI, ABEO, INGN)",0.901459,Date 2023-01-03 -1.523086 2023-01-04 -0.59...,-0.912386,1.951507,-0.600552,Health-Care
99,"(AADI, ABUS, CLNN)",0.901455,Date 2023-01-03 -4.238370 2023-01-04 -4.78...,-12.142131,1.414610,8.056797,Health-Care
100,"(AADI, ABSI, BACK)",0.900727,Date 2023-01-03 -2.264695 2023-01-04 -1.87...,-2.577503,0.716822,0.800053,Health-Care
101,"(AADI, ACET, CARA)",0.900564,Date 2023-01-03 0.740728 2023-01-04 0.91...,-3.533487,0.935223,0.260230,Health-Care


In [ ]:
industrials_df = get_best_stocks(train_df, industrials_tickerlist, 10000)
industrials_df = apply_filter_conditions(industrials_df)
industrials_df = industrials_df.sort_values("rsqaured", ascending=False)
industrials_df = industrials_df.reset_index(drop=True)
industrials_df['sector'] = 'Industrials'
industrials_df

,tickers,rsqaured,spreads,param_0,param_1,param_2,sector
0,"(AAON, BNGO, OMIC)",0.937241,Date 2023-01-03 1.087514 2023-01-04 -0.42...,18.967566,-0.253643,3.756827,Industrials
1,"(AAON, AKYA, OMIC)",0.934191,Date 2023-01-03 4.708766 2023-01-04 2.22...,27.947142,-0.755802,7.148070,Industrials
2,"(AAON, CDXS, OMIC)",0.930082,Date 2023-01-03 4.570963 2023-01-04 0....,38.332528,-0.800621,12.534653,Industrials
3,"(AAON, AKYA, MNTS)",0.919976,Date 2023-01-03 7.543654 2023-01-04 10....,3.195395,-0.340248,5.336712,Industrials
4,"(AAON, BYU, LIN)",0.906579,Date 2023-01-03 -4.626775 2023-01-04 -0....,284.551574,1.786099,-0.844622,Industrials
5,"(AAON, AGRI, RETO)",0.903130,Date 2023-01-03 2.474114 2023-01-04 21....,-198.544212,2.699500,8.649561,Industrials


In [ ]:
consumer_df = get_best_stocks(train_df, consumer_tickerlist, 10000)
consumer_df = apply_filter_conditions(consumer_df)
consumer_df = consumer_df.sort_values("rsqaured", ascending=False)
consumer_df = consumer_df.reset_index(drop=True)
consumer_df['sector'] = 'Consumer-Discretionary'
consumer_df

,tickers,rsqaured,spreads,param_0,param_1,param_2,sector
0,"(AAL, AIEV, ULCC)",0.956761,Date 2023-01-03 -0.638954 2023-01-04 -0.22...,112.688006,0.626946,-10.799248,Consumer-Discretionary
1,"(AAL, ADSE, JBLU)",0.949529,Date 2023-01-03 0.429228 2023-01-04 0.43...,-1.391350,0.659676,-0.314051,Consumer-Discretionary
2,"(AAL, AIEV, JBLU)",0.946229,Date 2023-01-03 -0.478619 2023-01-04 -0.22...,38.860583,0.543081,-3.811475,Consumer-Discretionary
3,"(AAL, ALTS, JBLU)",0.931938,Date 2023-01-03 -0.236261 2023-01-04 -0.09...,-1.743776,0.460772,1.763380,Consumer-Discretionary
4,"(AAL, AMSC, JBLU)",0.930450,Date 2023-01-03 0.406392 2023-01-04 0.42...,-2.018564,0.709223,-0.257912,Consumer-Discretionary
5,"(AAL, ADSE, ULCC)",0.922391,Date 2023-01-03 2.065826 2023-01-04 1.77...,-1.786985,0.968295,-0.838307,Consumer-Discretionary
6,"(AAL, AMZN, LMB)",0.920968,Date 2023-01-03 0.794946 2023-01-04 2.46...,-18.843376,-0.825449,0.460669,Consumer-Discretionary
7,"(AAL, ALTS, ULCC)",0.919616,Date 2023-01-03 0.066192 2023-01-04 0.14...,-2.386614,0.397067,4.974181,Consumer-Discretionary
8,"(AAL, AKAM, MMYT)",0.914457,Date 2023-01-03 -1.019987 2023-01-04 -1.50...,-8.474903,-0.439389,0.499738,Consumer-Discretionary
9,"(AAL, AENT, JD)",0.913290,Date 2023-01-03 -0.916221 2023-01-04 6.54...,11.416074,1.054903,3.310479,Consumer-Discretionary


In [ ]:
prescreened_df = pd.concat([technology_df, consumer_df, finance_df, healthcare_df, industrials_df], ignore_index=True)
prescreened_df

,tickers,rsqaured,spreads,param_0,param_1,param_2,sector
0,"(AAOI, AAPL, TTWO)",0.941185,Date 2023-01-03 6.929834 2023-01-04 9.65...,6.877691,0.837044,0.701438,Technology
1,"(AAOI, AIRG, PHUN)",0.936302,Date 2023-01-03 -1.832283 2023-01-04 -0.45...,-10.648280,-0.638324,7.967737,Technology
2,"(AAOI, AAPL, LRCX)",0.930703,Date 2023-01-03 -5.346900 2023-01-04 -1....,41.781284,7.707764,2.905373,Technology
3,"(AAOI, AAPL, MSFT)",0.915438,Date 2023-01-03 25.766030 2023-01-04 12....,-34.524471,1.266108,1.966564,Technology
4,"(AAOI, ADBE, INTU)",0.914145,Date 2023-01-03 -14.589959 2023-01-04 -15....,280.351750,5.028369,0.344191,Technology
...,...,...,...,...,...,...,...
269,"(AAON, AKYA, OMIC)",0.934191,Date 2023-01-03 4.708766 2023-01-04 2.22...,27.947142,-0.755802,7.148070,Industrials
270,"(AAON, CDXS, OMIC)",0.930082,Date 2023-01-03 4.570963 2023-01-04 0....,38.332528,-0.800621,12.534653,Industrials
271,"(AAON, AKYA, MNTS)",0.919976,Date 2023-01-03 7.543654 2023-01-04 10....,3.195395,-0.340248,5.336712,Industrials
272,"(AAON, BYU, LIN)",0.906579,Date 2023-01-03 -4.626775 2023-01-04 -0....,284.551574,1.786099,-0.844622,Industrials


In [ ]:
prescreened_df.to_csv("triplets_after_coefficient_tests_revised.csv")